In [2]:
import pandas as pd

# Load the CSV files into DataFrames
df_sales = pd.read_csv('data/sales.csv')
df_products = pd.read_csv('data/products.csv')
df_orders = pd.read_csv('data/orders.csv')
df_customers = pd.read_csv('data/customers.csv')

# standardize all ids to zero indexing
df_sales["order_id"] = df_sales["order_id"]-1
df_orders["order_id"] = df_orders["order_id"]-1
df_orders["customer_id"] = df_orders["customer_id"]-1
df_customers["customer_id"] = df_customers["customer_id"]-1

df_products.rename(columns={'product_ID': 'product_id'}, inplace=True)

# print(df_sales.head())
# print(df_sales.shape)
# print(df_products.head())
# print(df_products.shape)
# print(df_orders.head())
# print(df_orders.shape)
# print(df_customers.head())
# print(df_customers.shape)

df_merged = pd.merge(df_sales, df_products, on='product_id', how='inner')
df_merged = pd.merge(df_merged, df_orders, on='order_id', how='inner')
df_merged = pd.merge(df_merged, df_customers, on='customer_id', how='inner')


print(df_merged.head())
df_merged.to_csv('data/merged_data_raw.csv', index=False)
print(df_merged.shape)

   sales_id  order_id  product_id  price_per_unit  quantity_x  total_price  \
0         0         0         218             106           2          212   
1         1         0         481             118           1          118   
2         2         0           2              96           3          288   
3         3         0        1002             106           2          212   
4         4         0         691             113           3          339   

  product_type  product_name size  colour  ...  order_date  delivery_date  \
0        Shirt      Chambray    L  orange  ...   2021-8-30     2021-09-24   
1       Jacket        Puffer    S  indigo  ...   2021-8-30     2021-09-24   
2        Shirt  Oxford Cloth    M     red  ...   2021-8-30     2021-09-24   
3     Trousers          Wool    M    blue  ...   2021-8-30     2021-09-24   
4       Jacket         Parka    S  indigo  ...   2021-8-30     2021-09-24   

      customer_name       gender  age               home_address zip

In [3]:
# # Check for missing values
# print(df_merged.isnull().sum())

# # Check data types
# print(df_merged.dtypes)

# data cleaning
df_merged = df_merged.drop_duplicates()

# Convert 'order_date' and 'delivery_date' to datetime
df_merged['order_date'] = pd.to_datetime(df_merged['order_date'])
df_merged['delivery_date'] = pd.to_datetime(df_merged['delivery_date'])

# Example of a new feature: Days to deliver
df_merged['days_to_deliver'] = (df_merged['delivery_date'] - df_merged['order_date']).dt.days

# drop customer full name
df_merged.drop(columns=['customer_name'], inplace=True)
# drop delivery date and turn order date to seasons, add time to deliver
df_merged.drop(columns=['delivery_date'], inplace=True)
df_merged['order_date'] = pd.to_datetime(df_merged['order_date'])
df_merged['season'] = df_merged['order_date'].dt.quarter
# From products.csv, drop description
df_merged.drop(columns=['description'], inplace=True)
# From sales.csv, drop total price because it’s linear combination of quantity and unit price 
df_merged.drop(columns=['total_price'], inplace=True)
df_merged.to_csv('data/merged_data.csv', index=False)

# Display the adjusted DataFrame
print(df_merged)
df_merged.to_csv('data/merged_data.csv', index=False)



      sales_id  order_id  product_id  price_per_unit  quantity_x product_type  \
0            0         0         218             106           2        Shirt   
1            1         0         481             118           1       Jacket   
2            2         0           2              96           3        Shirt   
3            3         0        1002             106           2     Trousers   
4            4         0         691             113           3       Jacket   
...        ...       ...         ...             ...         ...          ...   
4995      2605       521         433             107           3       Jacket   
4996      3422       689         919              96           3     Trousers   
4997      3420       689         454             107           1       Jacket   
4998      3421       689         296             105           3        Shirt   
4999      3423       689         304             105           1        Shirt   

      product_name size  co

In [22]:
# numerical_vars = df_merged[['price_per_unit', 'quantity_sold', 'total_price', 'payment', 'age', 'days_to_deliver']]

# # Calculate correlation matrix
# correlation_matrix = numerical_vars.corr()

# print("Correlation Matrix:")
# print(correlation_matrix)

# # todo: non numerical variables

# import seaborn as sns
# import matplotlib.pyplot as plt

# # Plot heatmap of the correlation matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
# plt.title('Correlation Matrix Heatmap')
# plt.show()



In [16]:

# import scipy.stats as stats
# from itertools import combinations

# # Load your data
# df = pd.read_csv('data/merged_data.csv')

# # some transformation ideas
# # Convert date to datetime and extract month
# if 'order_date' in df.columns:
#     df['order_date'] = pd.to_datetime(df['order_date'])
#     df['month'] = df['order_date'].dt.month

# # Categorize age into groups
# if 'age' in df.columns:
#     df['age_group'] = pd.cut(df['age'], bins=[0, 18, 35, 65, 100], labels=['Youth', 'Young Adult', 'Adult', 'Senior'])

# # ANOVA
# def perform_anova(df, numerical, categorical):
#     grouped = df.groupby(categorical)[numerical].apply(list)
#     f_val, p_val = stats.f_oneway(*grouped)
#     return f_val, p_val

# # Chi-square test
# def perform_chi_square(df, cat1, cat2):
#     table = pd.crosstab(df[cat1], df[cat2])
#     chi2, p = stats.chi2_contingency(table)[:2]
#     return chi2, p

# # ANOVA on price per unit across product types
# f_val, p_val = perform_anova(df, 'price_per_unit', 'product_type')
# print(f"ANOVA on price per unit across product types: F-value={f_val}, p-value={p_val}")

# # Chi-square for product type and demographic groups
# chi2, p = perform_chi_square(df, 'product_type', 'gender')
# print(f"Chi-square for product type and gender: Chi2={chi2}, p-value={p}")

# # Chi-square for product type across cities
# if 'city' in df.columns:
#     chi2, p = perform_chi_square(df, 'product_type', 'city')
#     print(f"Chi-square for product type and city: Chi2={chi2}, p-value={p}")

# ANOVA for monthly sales
if 'month' in df.columns:
    f_val, p_val = perform_anova(df, 'quantity_x', 'month')
    print(f"ANOVA for quantity sold across months: F-value={f_val}, p-value={p_val}")
# # ANOVA for monthly sales
# if 'month' in df.columns:
#     f_val, p_val = perform_anova(df, 'quantity_sold', 'month')
#     print(f"ANOVA for quantity sold across months: F-value={f_val}, p-value={p_val}")


C:\Users\david\AppData\Local\Temp\ipykernel_85140\2588481844.py:1: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.3)
  import scipy.stats as stats


ANOVA on price per unit across product types: F-value=158.66390457543474, p-value=1.558232496968769e-67
Chi-square for product type and gender: Chi2=19.01887688289363, p-value=0.16422920443332853
Chi-square for product type and city: Chi2=1193.6851198108263, p-value=0.5940681608239126
ANOVA for quantity sold across months: F-value=0.7068397334048305, p-value=0.7032156607390802


In [ ]:
import numpy as np
from plotly import express as px

ids, id_counts = np.unique(df["product_id"], return_counts=True)

fig = px.violin(df["product_id"], points="all")
fig.show()

ages, age_counts = np.unique(df["age"], return_counts=True)

fig = px.violin(df["age"], points="all")
fig.show()

genders, gender_counts = np.unique(df["gender"], return_counts=True)

fig = px.violin(df["gender"], points="all")
fig.show()




In [9]:
import numpy as np

num_customers = df_customers.shape[0]
num_products = df_products.shape[0]

print(num_customers, "customers and", num_products, "products")

# user_item_matrix = np.empty((num_customers, num_products))
# user_item_matrix.fill(np.nan)

# # display(df_sales.head())

# for i in range(df_sales.shape[0]):
#     order_id = df_sales["order_id"][i]
#     customer_id = df_orders["customer_id"][order_id]
#     product_id = df_sales["product_id"][i]
#     quantity = df_sales["quantity"][i]

#     if np.isnan(user_item_matrix[customer_id, product_id]):
#         user_item_matrix[customer_id, product_id] = quantity
#     else:
#         user_item_matrix[customer_id, product_id] += quantity

# print("created user item matrix with shape", user_item_matrix.shape)

# print("sum of quantities in sales.csv is", np.sum(df_sales["quantity"]))
# print("sum of user item is", np.nansum(user_item_matrix))

# print("cutting empty rows")

# user_sums = np.nansum(user_item_matrix, axis=1)
# us_numbers, us_counts = np.unique(user_sums, return_counts=True)
# print(us_counts[0], "users with no purchases")

# user_item_matrix = user_item_matrix[~np.all(np.isnan(user_item_matrix), axis=1)]

# print("cutting empty columns")

# product_sums = np.nansum(user_item_matrix, axis=0)
# ps_numbers, ps_counts = np.unique(product_sums, return_counts=True)
# print(ps_counts[0], "unpurchased products")

# user_item_matrix = user_item_matrix[:, ~np.all(np.isnan(user_item_matrix[..., :]), axis=0)]

# print("user item matrix now has shape", user_item_matrix.shape)

# quantity = np.nansum(user_item_matrix)

# print("quantity is still", quantity)

# qs_numbers, qs_counts = np.unique(user_item_matrix, return_counts=True)
# print(qs_counts[-1], "empty cells")
# print(1-(qs_counts[-1]/(user_item_matrix.shape[0]*user_item_matrix.shape[1])), "percent of cells filled")

user_item_matrix = np.zeros((num_customers, num_products))

# display(df_sales.head())

for i in range(df_sales.shape[0]):
    order_id = df_sales["order_id"][i]
    customer_id = df_orders["customer_id"][order_id]
    product_id = df_sales["product_id"][i]
    quantity = df_sales["quantity"][i]

    if user_item_matrix[customer_id, product_id] == 0:
        user_item_matrix[customer_id, product_id] = quantity
    else:
        user_item_matrix[customer_id, product_id] += quantity

print("created user item matrix with shape", user_item_matrix.shape)

print("sum of quantities in sales.csv is", np.sum(df_sales["quantity"]))
print("sum of user item is", np.sum(user_item_matrix))

print("cutting empty rows")

user_sums = np.sum(user_item_matrix, axis=1)
us_numbers, us_counts = np.unique(user_sums, return_counts=True)
print(us_counts[0], "users with no purchases")

user_item_matrix = user_item_matrix[~np.all(user_item_matrix==0, axis=1)]

print("cutting empty columns")

product_sums = np.sum(user_item_matrix, axis=0)
ps_numbers, ps_counts = np.unique(product_sums, return_counts=True)
print(ps_counts[0], "unpurchased products")

user_item_matrix = user_item_matrix[:, ~np.all(user_item_matrix[..., :]==0, axis=0)]

print("user item matrix now has shape", user_item_matrix.shape)

quantity = np.sum(user_item_matrix)

print("quantity is still", quantity)

qs_numbers, qs_counts = np.unique(user_item_matrix, return_counts=True)
print(qs_counts[0], "empty cells")
print(1-(qs_counts[0]/(user_item_matrix.shape[0]*user_item_matrix.shape[1])), "percent of cells filled")




1000 customers and 1260 products
created user item matrix with shape (1000, 1260)
sum of quantities in sales.csv is 9962
sum of user item is 9962.0
cutting empty rows
384 users with no purchases
cutting empty columns
27 unpurchased products
user item matrix now has shape (616, 1233)
quantity is still 9962.0
754545 empty cells
0.006560653458463683 percent of cells filled
